## todo:
* Add them here

# General EDA
EDA playround to start to find interesting things.

In [6]:
# Import the required libraries
import pandas as pd
import numpy as np

# set up pandas to display floats in a more human friendly way
pd.options.display.float_format = '{:,.2f}'.format

In [7]:
df = pd.read_csv('../data/proc_kc_house_data.csv')
df.head(10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,lat,long,sqft_living15,sqft_lot15,date_month,date_day,date_year,renovated_status,sqft_living15_diff,sqft_lot15_diff
0,7129300520,10/13/2014,"221,900.00",3,1.00,1180,5650,1.00,0.00,0.00,...,47.51,-122.26,1340,5650,10,13,2014,UNK,-160,0
1,6414100192,12/9/2014,"538,000.00",3,2.25,2570,7242,2.00,0.00,0.00,...,47.72,-122.32,1690,7639,12,9,2014,Y,880,-397
2,5631500400,2/25/2015,"180,000.00",2,1.00,770,10000,1.00,0.00,0.00,...,47.74,-122.23,2720,8062,2,25,2015,UNK,-1950,1938
3,2487200875,12/9/2014,"604,000.00",4,3.00,1960,5000,1.00,0.00,0.00,...,47.52,-122.39,1360,5000,12,9,2014,UNK,600,0
4,1954400510,2/18/2015,"510,000.00",3,2.00,1680,8080,1.00,0.00,0.00,...,47.62,-122.05,1800,7503,2,18,2015,UNK,-120,577
5,7237550310,5/12/2014,"1,230,000.00",4,4.50,5420,101930,1.00,0.00,0.00,...,47.66,-122.00,4760,101930,5,12,2014,UNK,660,0
6,1321400060,6/27/2014,"257,500.00",3,2.25,1715,6819,2.00,0.00,0.00,...,47.31,-122.33,2238,6819,6,27,2014,UNK,-523,0
7,2008000270,1/15/2015,"291,850.00",3,1.50,1060,9711,1.00,0.00,0.00,...,47.41,-122.31,1650,9711,1,15,2015,UNK,-590,0
8,2414600126,4/15/2015,"229,500.00",3,1.00,1780,7470,1.00,0.00,0.00,...,47.51,-122.34,1780,8113,4,15,2015,UNK,0,-643
9,3793500160,3/12/2015,"323,000.00",3,2.50,1890,6560,2.00,0.00,0.00,...,47.37,-122.03,2390,7570,3,12,2015,UNK,-500,-1010


# Do basic data investigation
* Describe the data - mean, median, min, max
* Look for outliers

In [11]:
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,zipcode,lat,long,sqft_living15,sqft_lot15,date_month,date_day,date_year,sqft_living15_diff,sqft_lot15_diff
count,"21,597.00","21,597.00","21,597.00","21,597.00","21,597.00","21,597.00","21,597.00","21,597.00","21,597.00","21,597.00",...,"21,597.00","21,597.00","21,597.00","21,597.00","21,597.00","21,597.00","21,597.00","21,597.00","21,597.00","21,597.00"
mean,"4,580,474,287.77","540,296.57",3.37,2.12,"2,080.32","15,099.41",1.49,0.01,0.23,3.41,...,"98,077.95",47.56,-122.21,"1,986.62","12,758.28",6.57,15.69,"2,014.32",93.70,"2,341.13"
std,"2,876,735,715.75","367,368.14",0.93,0.77,918.11,"41,412.64",0.54,0.08,0.76,0.65,...,53.51,0.14,0.14,685.23,"27,274.44",3.12,8.63,0.47,600.61,"28,921.78"
min,"1,000,102.00","78,000.00",1.00,0.50,370.00,520.00,1.00,0.00,0.00,1.00,...,"98,001.00",47.16,-122.52,399.00,651.00,1.00,1.00,"2,014.00","-2,310.00","-326,879.00"
25%,"2,123,049,175.00","322,000.00",3.00,1.75,"1,430.00","5,040.00",1.00,0.00,0.00,3.00,...,"98,033.00",47.47,-122.33,"1,490.00","5,100.00",4.00,8.00,"2,014.00",-220.00,-421.00
50%,"3,904,930,410.00","450,000.00",3.00,2.25,"1,910.00","7,618.00",1.50,0.00,0.00,3.00,...,"98,065.00",47.57,-122.23,"1,840.00","7,620.00",6.00,16.00,"2,014.00",0.00,0.00
75%,"7,308,900,490.00","645,000.00",4.00,2.50,"2,550.00","10,685.00",2.00,0.00,0.00,4.00,...,"98,118.00",47.68,-122.12,"2,360.00","10,083.00",9.00,23.00,"2,015.00",310.00,680.00
max,"9,900,000,190.00","7,700,000.00",33.00,8.00,"13,540.00","1,651,359.00",3.50,1.00,4.00,5.00,...,"98,199.00",47.78,-121.31,"6,210.00","871,200.00",12.00,31.00,"2,015.00","8,690.00","1,225,778.00"


# Framing questions from the POV of County government officals.  

Residential property tax assessors, county/city planning that want to learn about economic factors/data related to Residential property. Here are examples;
* Does my zip code price out residents? Ddoes the city/county need to work on making homes more affordable...are they already affordable?
* are lots too small/overcrowding/stressing infrastucture?  Are there differences in lot size between zip codes?
* are the homes good quality, newer, renovated if older. Are there differences between zip codes.
* for tax purposes what percentage of homes are being renovated? What types of homes are being renovated? 

## General Questions
### Are any homes listed twice in the dataset?
I think only home sales in 2014 and 2015 are included so there might not be many homes that sold twice in that time frame.

### 

## What percentage of homes in the county are renovated? Consider breaking down by zipcode, by year built, by homes with waterfront and/or nice views.

## Group by zipcode and look at the differences in 1)sales price 2)sqft_living 3) sqft_lot 4)yr_built  5)anything else interesting.

## Exploring home living area and lot compared to neighbors 


In [10]:
print("--- df shape---")
print(df.shape)
print("--- no difference between home/lot and neighbors ---")
print(df[df['sqft_living15_diff'] == 0].shape)
print(df[df['sqft_lot15_diff'] == 0].shape)
print("---- home/lot LARGER than neighbors---")
print(df[df['sqft_living15_diff'] > 0].shape)
print(df[df['sqft_lot15_diff']>0].shape)
print("---- home/lot SMALLER than neighbors---")
print(df[df['sqft_living15_diff'] < 0].shape)
print(df[df['sqft_lot15_diff']<0].shape)

--- df shape---
(21597, 27)
--- no difference between home/lot and neighbors ---
(2563, 27)
(4474, 27)
---- home/lot LARGER than neighbors---
(9838, 27)
(8593, 27)
---- home/lot SMALLER than neighbors---
(9196, 27)
(8530, 27)


In [ ]:

print("----renovated status UNK, N, Y---")
print(df[df['renovated_status'] == 'UNK'].shape)
print(df[df['renovated_status'] == 'N'].shape)
print(df[df['renovated_status'] == 'Y'].shape)
print('--- different sqft_living15 and UNK renovated_status------')
print(df[(df['sqft_living15_different'] == 'Y') & (df['renovated_status']=='UNK')].shape)